In [1]:
import nilearn
from nilearn import image
import os
import nibabel as nib
import numpy as np
import pandas as pd
from nilearn.image import load_img
from nilearn.image import math_img
from nilearn.image import resample_to_img

In [2]:
#resample all subjects and hotspots
directory = [i for i in os.listdir('/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/Efields') if i.endswith('hemi-L.nii')]

#resample patients
def resample(sub,stn,path):
    re_sub = resample_to_img(sub,stn,interpolation='nearest')
    nib.save(re_sub,path)

#load in STNs as anchor
stn_lh = load_img('/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/HotspotAtlas/resampled/STN_motor_lh.nii.gz')
stn_rh = load_img('/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/HotspotAtlas/resampled/STN_motor_rh.nii.gz')

#resample hotspot
hotspot_lh = load_img('/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/HotspotAtlas/TORPSM_hotspot_lh.nii.gz')
hotspot_rh = load_img('/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/HotspotAtlas/TORPSM_hotspot_rh.nii.gz')

resample(hotspot_lh,stn_lh,'/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/HotspotAtlas/resampled/TORPSM_hotspot_lh.nii.gz')
resample(hotspot_rh,stn_rh,'/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/HotspotAtlas/resampled/TORPSM_hotspot_rh.nii.gz')

#resample subjects
for i in directory:
    #load in patients
    lh_sub = load_img(f'/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/Efields/{i[:-5]}L.nii')
    rh_sub = load_img(f'/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/Efields/{i[:-5]}R.nii')

    #call resample
    resample(lh_sub,stn_lh,f'/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/Efields/resampled/{i[:-5]}L.nii')
    resample(rh_sub,stn_rh,f'/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/Efields/resampled/{i[:-5]}R.nii')

In [3]:
#get volumes of overlap
#make lists to store outputs in
stn_lh_vol,stn_rh_vol,hotspot_lh_vol,hotspot_rh_vol = [],[],[],[]

#load in stn & hotspot
stn_lh = load_img('/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/HotspotAtlas/resampled/STN_motor_lh.nii.gz')
stn_rh = load_img('/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/HotspotAtlas/resampled/STN_motor_rh.nii.gz')
hotspot_lh = load_img('/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/HotspotAtlas/resampled/TORPSM_hotspot_lh.nii.gz')
hotspot_rh = load_img('/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/HotspotAtlas/resampled/TORPSM_hotspot_rh.nii.gz')
    
def get_efield_overlap(stn,sub,volume_list):
    mask = math_img('img1*img2',img1=stn,img2=sub) #mask subject and stn/hotspot so that only the overlapped voxels remain
    mask = mask.get_fdata()
    no0_mask = mask[mask!=0] #take out all 0s so they aren't included in the mean
    volume = np.mean(no0_mask) #take average of overlapped voxels
    volume_list.append(volume) #add to list

#loop over to get volume
for i in directory:
    #load in subjects
    sub_lh = load_img(f'/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/Efields/resampled/{i[:-5]}L.nii')
    sub_rh = load_img(f'/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/Efields/resampled/{i[:-5]}R.nii')

    #call the function
    get_efield_overlap(stn_lh,sub_lh,stn_lh_vol)
    get_efield_overlap(stn_rh,sub_rh,stn_rh_vol)
    get_efield_overlap(hotspot_lh,sub_lh,hotspot_lh_vol)
    get_efield_overlap(hotspot_rh,sub_rh,hotspot_rh_vol)

In [4]:
#get bilateral values for each column, taking the average
#make lists to store outputs in
stn_bi_vol,hotspot_bi_vol = [],[]
    
def efield_bilateral(list1,list2,averages_list): #take average of each subject in lh and rh lists
    averages = [(x + y) / 2 for x, y in zip(list1, list2)]
    averages_list.extend(averages)

efield_bilateral(stn_lh_vol,stn_rh_vol,stn_bi_vol)
efield_bilateral(hotspot_lh_vol,hotspot_rh_vol,hotspot_bi_vol)

In [5]:
#get ids of subjects
sub_ids = []
for i in directory:
    sub_ids.append(i[:-35])

In [8]:
#export into csv
df = pd.DataFrame(
    {'ID': sub_ids,
     'Motor_STN_LH_Volume_Overlap': stn_lh_vol,
     'Motor_STN_RH_Volume_Overlap': stn_rh_vol,
     'Motor_STN_Bi_Volume_Overlap': stn_bi_vol,
     'TOR-PSM_STN_Sweetspot_LH_Volume_Overlap': hotspot_lh_vol,
     'TOR-PSM_STN_Sweetspot_RH_Volume_Overlap': hotspot_rh_vol,
     'TOR-PSM_STN_Sweetspot_Bi_Volume_Overlap': hotspot_bi_vol
    })

df.to_csv('/Volumes/Cingulate/NativeMNI_Overlaps/Bahne/Volume_Overlap.csv')